In [ ]:


import sklearn
import pandas as pd  
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.



In [5]:
df= pd.read_csv('used_cars.csv')
y = df['price']
x = df.drop('price', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Train size:", x_train.shape, y_train.shape)
print("Test size:", x_test.shape, y_test.shape)


Train size: (3207, 11) (3207,)
Test size: (802, 11) (802,)
